##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 将 SessionRunHook 迁移到 Keras 回调

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://tensorflow.google.cn/guide/migrate/sessionrunhook_callback"><img src="https://tensorflow.google.cn/images/tf_logo_32px.png">在 TensorFlow.org 上查看</a>   </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/guide/migrate/sessionrunhook_callback.ipynb"><img src="https://tensorflow.google.cn/images/colab_logo_32px.png">在 Google Colab 运行</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/guide/migrate/sessionrunhook_callback.ipynb"><img src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png">在 Github 上查看源代码</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/guide/migrate/sessionrunhook_callback.ipynb"><img src="https://tensorflow.google.cn/images/download_logo_32px.png">下载笔记本</a>   </td>
</table>


在 TensorFlow 1 中，要自定义训练的行为，可以使用 `tf.estimator.SessionRunHook` 和 `tf.estimator.Estimator`。本指南演示了如何使用 `tf.keras.callbacks.Callback` API 从 `SessionRunHook` 迁移到 TensorFlow 2 的自定义回调，此 API 与 Keras `Model.fit` 一起用于训练（以及 `Model.evaluate` 和 `Model.predict`）。您将通过实现 `SessionRunHook` 和 `Callback` 任务来学习如何做到这一点，此任务会在训练期间测量每秒的样本数。

回调的示例为检查点保存 (`tf.keras.callbacks.ModelCheckpoint`) 和 [TensorBoard](%60tf.keras.callbacks.TensorBoard%60) 摘要编写。Keras [回调](https://tensorflow.google.cn/guide/keras/custom_callback)是在内置 Keras `Model.fit`/`Model.evaluate`/`Model.predict` API 中的训练/评估/预测期间的不同点调用的对象。可以在 `tf.keras.callbacks.Callback` API 文档以及[编写自己的回调](https://tensorflow.google.cn/guide/keras/custom_callback.ipynb/)和[使用内置方法进行训练和评估](https://tensorflow.google.cn/guide/keras/train_and_evaluate)（*使用回调*部分）指南中详细了解回调。

## 安装

从导入和用于演示目的的简单数据集开始：

In [ ]:
import tensorflow as tf
import tensorflow.compat.v1 as tf1

import time
from datetime import datetime
from absl import flags

In [ ]:
features = [[1., 1.5], [2., 2.5], [3., 3.5]]
labels = [[0.3], [0.5], [0.7]]
eval_features = [[4., 4.5], [5., 5.5], [6., 6.5]]
eval_labels = [[0.8], [0.9], [1.]]

## TensorFlow 1：使用 tf.estimator API 创建自定义 SessionRunHook

下面的 TensorFlow 1 示例展示了如何设置自定义 `SessionRunHook` 以在训练期间测量每秒的样本数。创建钩子 (`LoggerHook`) 后，将其传递给 `tf.estimator.Estimator.train` 的 `hooks` 参数。

In [ ]:
def _input_fn():
  return tf1.data.Dataset.from_tensor_slices(
      (features, labels)).batch(1).repeat(100)

def _model_fn(features, labels, mode):
  logits = tf1.layers.Dense(1)(features)
  loss = tf1.losses.mean_squared_error(labels=labels, predictions=logits)
  optimizer = tf1.train.AdagradOptimizer(0.05)
  train_op = optimizer.minimize(loss, global_step=tf1.train.get_global_step())
  return tf1.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

In [ ]:
class LoggerHook(tf1.train.SessionRunHook):
  """Logs loss and runtime."""

  def begin(self):
    self._step = -1
    self._start_time = time.time()
    self.log_frequency = 10

  def before_run(self, run_context):
    self._step += 1

  def after_run(self, run_context, run_values):
    if self._step % self.log_frequency == 0:
      current_time = time.time()
      duration = current_time - self._start_time
      self._start_time = current_time
      examples_per_sec = self.log_frequency / duration
      print('Time:', datetime.now(), ', Step #:', self._step,
            ', Examples per second:', examples_per_sec)

estimator = tf1.estimator.Estimator(model_fn=_model_fn)

# Begin training.
estimator.train(_input_fn, hooks=[LoggerHook()])

## TensorFlow 2：为 Model.fit 创建自定义 Keras 回调

在 TensorFlow 2 中，当您使用内置 Keras `Model.fit`（或 `Model.evaluate`）进行训练/评估时，可以配置自定义 `tf.keras.callbacks.Callback`，然后将其传递给 `Model.fit`（或 `Model.evaluate`）的 `callbacks` 参数。（在[编写自己的回调](../..guide/keras/custom_callback.ipynb)指南中了解详情。）

在下面的示例中，您将编写一个自定义 `tf.keras.callbacks.Callback` 来记录各种指标 – 它将测量每秒的样本数，这应该与前面的 `SessionRunHook` 示例中的指标相当。

In [ ]:
class CustomCallback(tf.keras.callbacks.Callback):

    def on_train_begin(self, logs = None):
      self._step = -1
      self._start_time = time.time()
      self.log_frequency = 10

    def on_train_batch_begin(self, batch, logs = None):
      self._step += 1

    def on_train_batch_end(self, batch, logs = None):
      if self._step % self.log_frequency == 0:
        current_time = time.time()
        duration = current_time - self._start_time
        self._start_time = current_time
        examples_per_sec = self.log_frequency / duration
        print('Time:', datetime.now(), ', Step #:', self._step,
              ', Examples per second:', examples_per_sec)

callback = CustomCallback()

dataset = tf.data.Dataset.from_tensor_slices(
    (features, labels)).batch(1).repeat(100)

model = tf.keras.models.Sequential([tf.keras.layers.Dense(1)])
optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.05)

model.compile(optimizer, "mse")

# Begin training.
result = model.fit(dataset, callbacks=[callback], verbose = 0)
# Provide the results of training metrics.
result.history

## 后续步骤

通过下列方式详细了解回调：

- API 文档：`tf.keras.callbacks.Callback`
- 指南：[编写自己的回调](../..guide/keras/custom_callback.ipynb/)
- 指南：[使用内置方法进行训练和评估](https://tensorflow.google.cn/guide/keras/train_and_evaluate)（*使用回调*部分）

此外，您可能还会发现下列与迁移相关的资源十分有用：

- [提前停止迁移指南](early_stopping.ipynb)：`tf.keras.callbacks.EarlyStopping` 是一个内置的提前停止回调
- [TensorBoard 迁移指南](tensorboard.ipynb)：TensorBoard 支持跟踪和显示指标
- [LoggingTensorHook 和 StopAtStepHook 到 Keras 回调迁移指南](logging_stop_hook.ipynb)